In [1]:
import pandas as pd
import numpy as np
import tushare as ts

In [2]:
code = '002398'  # 股票代码
body_size = 0.03  # 锤子实体上限
head_size = 0.5   # 锤子上影线长度
tail_size = 2    # 锤子下影线长度
length = 10     # 观察期时间长短
stoplose_trigger = 1  # 当价格偏离均线满足几倍标准差时止损

In [3]:
data = ts.get_k_data(code,'2012-01-01')

data.sort_index(ascending=True,inplace=True)

data.head()

,date,open,close,high,low,volume,code
0,2012-01-04,3.521,3.396,3.605,3.392,2834.0,002398
1,2012-01-05,3.383,3.057,3.396,3.057,8209.0,002398
2,2012-01-06,3.027,3.022,3.094,2.882,9726.0,002398
3,2012-01-09,3.020,3.152,3.165,2.953,5365.0,002398
4,2012-01-10,3.152,3.292,3.325,3.152,11215.0,002398


In [4]:
data['pct_change'] = data['close'].pct_change()

In [5]:
data['ma'] = data['close'].rolling(length).mean()

In [6]:
data['std'] = data['close'].rolling(length).std()

In [7]:
del data['volume']

In [8]:
data.tail()

,date,open,close,high,low,code,pct_change,ma,std
1837,2020-01-23,6.83,6.53,6.85,6.43,002398,-0.042522,6.754,0.110675
1838,2020-02-03,5.88,5.88,6.10,5.88,002398,-0.099541,6.683,0.297547
1839,2020-02-04,5.79,6.01,6.13,5.79,002398,0.022109,6.605,0.361701
1840,2020-02-05,6.03,6.03,6.20,5.98,002398,0.003328,6.528,0.395918
1841,2020-02-06,6.05,6.07,6.13,5.95,002398,0.006633,6.453,0.405382


In [9]:
# 计算实盘中的移动平均
data['yes_ma'] = data['ma'].shift(1)
data['yes_std'] = data['std'].shift(1)

In [10]:
# 识别锤子形态
# 计算实体，上影线，下影线
data['body'] = abs(data['open'] - data['close'])

# 计算上影线
data['head'] = data['high'] - data[['open','close']].max(axis=1)

# 计算下影线
data['tail'] = data[['open','close']].min(axis=1) - data['low']

In [11]:
data.head()

,date,open,close,high,low,code,pct_change,ma,std,yes_ma,yes_std,body,head,tail
0,2012-01-04,3.521,3.396,3.605,3.392,002398,NaN,NaN,NaN,NaN,NaN,0.125,0.084,0.004
1,2012-01-05,3.383,3.057,3.396,3.057,002398,-0.099823,NaN,NaN,NaN,NaN,0.326,0.013,0.000
2,2012-01-06,3.027,3.022,3.094,2.882,002398,-0.011449,NaN,NaN,NaN,NaN,0.005,0.067,0.140
3,2012-01-09,3.020,3.152,3.165,2.953,002398,0.043018,NaN,NaN,NaN,NaN,0.132,0.013,0.067
4,2012-01-10,3.152,3.292,3.325,3.152,002398,0.044416,NaN,NaN,NaN,NaN,0.140,0.033,0.000


In [12]:
# 判断K线各部分是否符合锤子线要求
data['body_cond'] = np.where(data['body']/data['open'] < body_size,1,0)  # 实体大小比开盘价要小于3%，k线实体

In [13]:
# 下影线比实体的
data['head_cond'] = np.where(data['tail'] == 0,False,data['head']/data['tail'] < head_size)

In [14]:
data['tail_cond'] = np.where(data['body'] == 0,True,data['tail']/data['body'] > tail_size)

In [15]:
data.head()

,date,open,close,high,low,code,pct_change,ma,std,yes_ma,yes_std,body,head,tail,body_cond,head_cond,tail_cond
0,2012-01-04,3.521,3.396,3.605,3.392,002398,NaN,NaN,NaN,NaN,NaN,0.125,0.084,0.004,0,False,False
1,2012-01-05,3.383,3.057,3.396,3.057,002398,-0.099823,NaN,NaN,NaN,NaN,0.326,0.013,0.000,0,False,False
2,2012-01-06,3.027,3.022,3.094,2.882,002398,-0.011449,NaN,NaN,NaN,NaN,0.005,0.067,0.140,1,True,True
3,2012-01-09,3.020,3.152,3.165,2.953,002398,0.043018,NaN,NaN,NaN,NaN,0.132,0.013,0.067,0,True,False
4,2012-01-10,3.152,3.292,3.325,3.152,002398,0.044416,NaN,NaN,NaN,NaN,0.140,0.033,0.000,0,False,False


In [16]:
data['hammer'] = data[['head_cond','body_cond','tail_cond']].all(axis=1)

In [17]:
data[data['hammer']].tail(5)

,date,open,close,high,low,code,pct_change,ma,std,yes_ma,yes_std,body,head,tail,body_cond,head_cond,tail_cond,hammer
1786,2019-11-12,5.32,5.28,5.35,5.18,002398,-0.009381,5.375,0.076194,5.397,0.077467,0.04,0.03,0.10,1,True,True,True
1799,2019-11-29,5.35,5.35,5.37,5.30,002398,0.009434,5.359,0.045080,5.336,0.088217,0.00,0.02,0.05,1,True,True,True
1810,2019-12-16,5.79,5.80,5.80,5.72,002398,0.003460,5.648,0.149800,5.609,0.156450,0.01,0.00,0.07,1,True,True,True
1816,2019-12-24,5.75,5.76,5.78,5.69,002398,-0.001733,5.782,0.042111,5.784,0.041419,0.01,0.02,0.06,1,True,True,True
1821,2019-12-31,6.05,6.04,6.05,5.95,002398,0.003322,5.862,0.100532,5.841,0.078804,0.01,0.00,0.09,1,True,True,True


In [18]:
# 昨天的hammer数据
data['yes_hammer'] = data['hammer'].shift(1)

### 编写交易逻辑 - 循环法

In [ ]:
flag = 0  # 持仓记录，1：有仓位，0：空仓

for i in range(2*length,len(data)):   # 从20天开始计算，因为前期数据无效
    # 如果已持仓，判断是否止损
    if flag == 1:
        stoplose_price = max(data.loc[i,'yes_ma'] - stoplose_trigger * data.loc[i,'yes_std'],long)
        # 当天价格低于止损价，则进行止损，一个是移动止损，一个是开仓时候的开仓和开仓价-1倍标准差
        if data.loc[i,'low'] < stoplose_price:
            flag = 0
            data.loc[i,'return']  = min(data.loc[i,'open'],stoplose_price) / data.loc[i-1,'close'] - 1
            data.loc[i,'trade_mark'] = -10  
        else:
            pass
            